## Install

In [ ]:
pip install --upgrade azure-ai-ml

## Run job

In [ ]:
# This scripts starts a job on a cluster.
# Run this script to run the source code in ./src/main.py
#
from azure.ai.ml import MLClient, command, Input, Output
from azure.identity import DefaultAzureCredential

# Create the ML Client
ml_client = MLClient.from_config(DefaultAzureCredential())


### 1. Run preMEDS.py

### 2. Run MEDS

In [ ]:
# Prepare input directories
inp_uri = f'azureml://subscriptions/f8c5aac3-29fc-4387-858a-1f61722fb57a/resourcegroups/forskerpl-n0ybkr-rg/workspaces/forskerpl-n0ybkr-mlw/datastores/workspaceblobstore/paths/MEDS_test/data/preMEDS'
inputs = {"input_dir": Input(type="uri_folder", path=inp_uri)}

# Prepare output directories
out_uri = f'azureml://subscriptions/f8c5aac3-29fc-4387-858a-1f61722fb57a/resourcegroups/forskerpl-n0ybkr-rg/workspaces/forskerpl-n0ybkr-mlw/datastores/workspaceblobstore/paths/MEDS_test/data/MEDS_data/'
outputs = {"output_dir":Output(path=out_uri, type="uri_folder", mode="rw_mount")}

In [ ]:

# Chose a compute cluster target (review the options at "Compute" => "Compute clusters")
compute_target = "CPU-20-LP"

# Start the job
job = command(
    code=".", # Folder with source code
    command="bash run_MEDS_transform.sh ${{inputs.input_dir}}  ${{inputs.input_dir}} ${{outputs.output_dir}}",
    inputs=inputs,
    outputs=outputs,
    environment="MEDS_transform@latest",
    compute=compute_target,
)

returned_job = ml_client.create_or_update(job, experiment_name="MEDS")

## Make env

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment

mlc = MLClient.from_config(DefaultAzureCredential())

env = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="../envs/MEDS_transform.yaml",
    name="MEDS_transform",
    description="Environment for MEDS-transforms",
)
mlc.environments.create_or_update(env) 